In [1]:
import pandas as pd
import json
from itertools import chain

file_couples = [
    # "js_2023_02_06/warm/query_responses.json",
    # "js_2023_02_06/cold/query_responses.json",
    # "merlin/cmi-cache-only/all-commands/main/full_responses.json",
    # # "merlin/cmi-cache-only/all-commands/merlin-domains/full_responses.json",
    # # "merlin/cmi-cache-only/complete-prefix/main/full_responses.json",
    # # "merlin/cmi-cache-only/complete-prefix/merlin-domains/full_responses.json",
    # "merlin/full-cache/all-commands/main/full_responses.json",
    # "merlin/full-cache/all-commands/merlin-domains/full_responses.json",
    # "irmin-15-10-2025/10/cmi-cache-only/all-commands/data-race-free-merlin-domains",
    # "irmin-15-10-2025/10/cmi-cache-only/all-commands/main",
    [
        ("ocaml-lsp-17-10-2025/src/10/cmi-cache-only/all-commands/main/full_responses.json", "Ocaml-lsp. Mixed hot/cold cache", "merlin"),
        ("ocaml-lsp-17-10-2025/src/10/cmi-cache-only/all-commands/data-race-free-merlin-domains/full_responses.json", "Ocaml-lsp. Mixed hot/cold cache", "merlin-domains"),
    ],
    [
        ("irmin-16-10-2025/cmi-cache-only/irmin/all-commands/main/irmin/full_responses.json", "Irmin. Mixed hot/cold cache", "merlin"),
        ("irmin-16-10-2025/cmi-cache-only/irmin/all-commands/data-race-free-merlin-domains/irmin/full_responses.json", "Irmin. Mixed hot/cold cache", "merlin-domains"),
    ],
    [
        ("irmin-16-10-2025/full-cache/irmin/all-commands/main/full_responses.json", "Irmin. Hot cache", "merlin"),
        ("irmin-16-10-2025/full-cache/irmin/all-commands/data-race-free-merlin-domains/full_responses.json", "Irmin. Hot cache", "merlin-domains"),
    ],
    [
        ("ocaml-lsp-17-10-2025/types/type-enclosing-only/no-cache/main-merlin/full_responses.json", "Long file in OCamlLSP. Cold cache", "merlin"),
        ("ocaml-lsp-17-10-2025/types/type-enclosing-only/no-cache/data-race-free-merlin-domains/full_responses.json", "Long file in OCamlLSP. Cold cache", "merlin-domains"),
    ],



    
]


def frame_from_file(file):
    data = pd.read_json(file, lines = True)

    resp = list(chain.from_iterable(data['responses']))

    resp_with_id = []

    for _, row in data.iterrows():
        sample_id = row['sample_id']
        unix_time = row['unix_time']
        for resp in row['responses']:
            resp_copy = resp.copy()
            resp_copy['sample_id'] = sample_id
            resp_copy['unix_time'] = unix_time
            resp_with_id.append(resp_copy)

    return pd.json_normalize(resp_with_id, sep='.')


In [2]:
def summarize_unix(couple):
    percentiles = [0.25, 0.5, 0.75, 0.9, 0.95]
    rows = []

    for (file, desc, merlin) in couple:
        frame = frame_from_file(file)
        desc_unix = frame['unix_time'].describe(percentiles=percentiles)

                # Skip if describe() failed (e.g. column missing or empty)
        if desc_unix.isnull().all():
            print(f"⚠️ Skipping {file} — no valid unix_time data.")
            continue


        row = desc_unix.to_dict()  # convert Series → dict
        row['merlin'] = merlin  # label which file this came from
        rows.append(row)

    # Build a single DataFrame from all rows
    df = pd.DataFrame(rows)
    df = df.set_index('merlin')
    df = df.drop(columns=['min', 'max'])
    df.index.name = None
    df = df.round(1)
    print(desc)
    display(df)
    return df

for couple in file_couples:
    summarize_unix(couple)

Ocaml-lsp. Mixed hot/cold cache


,count,mean,std,25%,50%,75%,90%,95%
merlin,1591.0,84.3,288.6,12.1,23.7,36.3,53.8,99.2
merlin-domains,1591.0,78.7,286.4,13.0,20.8,34.2,48.9,87.0


Irmin. Mixed hot/cold cache


,count,mean,std,25%,50%,75%,90%,95%
merlin,4136.0,76.3,77.6,55.4,60.3,69.6,119.7,173.2
merlin-domains,4136.0,71.3,68.4,55.7,60.8,70.2,92.7,137.9


Irmin. Hot cache


,count,mean,std,25%,50%,75%,90%,95%
merlin,4136.0,63.2,58.3,53.7,58.9,67.1,75.5,93.9
merlin-domains,4136.0,58.7,57.2,46.0,58.4,62.3,71.7,85.9


Long file in OCamlLSP. Cold cache


,count,mean,std,25%,50%,75%,90%,95%
merlin,115.0,1559.1,104.9,1461.1,1493.7,1670.0,1681.6,1687.6
merlin-domains,115.0,1206.1,408.8,844.9,1203.2,1521.0,1770.3,1825.0
